### MQTT Blaster ###

In [4]:
#!/usr/bin/env python3
import paho.mqtt.client as mqtt
import time
from timeit import default_timer as timer
from datetime import timedelta
import multiprocessing
import numpy as np
import getopt, sys
import subprocess, os

# This is the Publisher
major_number=0
minor_number=1
version_info="mqtt-blaster: V"+str(major_number)+"."+str(minor_number)

# mqtt configurations
SERVER_IP = "192.168.29.149"
#SERVER_IP = "ec2-3-81-9-181.compute-1.amazonaws.com"
PORT = 1883
KEEP_ALIVE = 60
QOS0 = 0
QOS1 = 1
QOS2 = 2
RETAIN = False

QOS = QOS1

# number of topics/publishers
max_publishers = 3
max_topics = 1

topic_root = "topic/"
topic_latency = topic_root+"latency"
topic_summary = topic_root+"summary"

# approximate message rate. sleep time is 1 sec/rate (40000 is max rate)
# for loop constant is 35171223
msg_count = 350000
max_msg_rate = 40000
for_constant = 23101223
sleep_interval = int(for_constant/(max_msg_rate/max_publishers))

not_end = 1

def usage():
    print(version_info)
    print("\nUsage: $ mqtt-blaster [options]... -s [ipaddress/hostname]\n")
    print("Options:\n--------")
    print("-q, qos: [0, 1, 2], default: 0")
    print("-r, rate of message [1-6], default: 1 ")
    print("-c, message count [1-1000000], default: 1 ")
    print("-p, number of publishers to be used [1-4], default: 1")
    print("-t, number of topics [1-4], default: 1") 
    print("-v, print version info") 
    print("-h, print this help")
    print("-a, try remote/local performace script installation,[0,1] default: 0 (do not try)")
    print("    this option requires, keyless authentication enabled between client and server")
    print("    Note: enabling keyless authentication is an onetime activity 'and' is applicable to")
    print("          locally deployed servers. Public deployments may not allow ssh logins or ")
    print("          messuring server KPIs")
    print("Dependencies\n------------")
    print("Following tools are required on server and client(publisher) side")
    print("to measure various system parameters during experiment:")
    print("python 3.x, top, cpustat, a, b, c")
    pass

def on_publish_publisher(client, userdata, msg):
    #print("on_publish")
    pass

# this is the publisher thread
def publisher_process(publisher_id):
    print("Publisher [" + str(publisher_id) + "] started...\r")
    client = mqtt.Client()
    client.max_inflight_messages_set(0)
    client.on_publish = on_publish_publisher
    client.connect(SERVER_IP, PORT, KEEP_ALIVE)

    # time to settle down
    time.sleep(2)

    # setup message
    #message_prefix = "PUB" + str(publisher_id) + "." + topic_root
    message_prefix = "PUB" + str(publisher_id) + "." + topic_root + str(publisher_id) + ": m#"
    #padding = ".Ac#4d%^NHTyAUDkajYRASIKDag98124326810: m#"
    padding = ".Ac#4d%^NHTyAUDkajYRASIKDag98124326810: m#"
    message_prefix += padding
    
    #client.loop_start()
    # exp start time
    start = timer()
   
    # each publisher publishes 'msg_count' number of messages per topic
    #changing to topic per publishe
    topic = topic_root + str(publisher_id)
    ###message_no = 0
    
    ### while(message_no < msg_count):
        ###latency_calc_sub.loop_start()
        ###message = message_prefix + str(message_no) + "\r"
        ###client.publish(topic, message, qos=QOS, retain=RETAIN)
        ###message_no += 1
        
    ###client.loop_start()
    
    for message_no in range(msg_count):
        #for topic_id in range(max_topics):
            # prepare topic and message
            #topic = topic_root + str(topic_id)
            #message = message_prefix + str(topic_id) + "." + str(message_no) + ":" +str(timer()) + "\r"
        #message = message_prefix + "." + str(message_no) + ":" +str(timer()) + "\r"
        message = message_prefix + str(message_no) + "\r"
            
            # publish the message
        client.publish(topic, message, qos=QOS, retain=RETAIN)
        # time to sleep/rate control. (below)
        # number to get 100% cpu utilization on server.
        # mqtt: qos1: , qos2: 1000
        #for i in range (10000):
        #    pass
        time.sleep(0.0001)

    # exp end time
    end = timer()
    elapsed_time = end -start

    # now publish the numbers: each publisher publishes total number messages sent on all topics
    # currently publishing summary on topic_summary channel. 
    time.sleep(2)
    message = "\nSummary\n-------\nPublisher: " + str(publisher_id)+"\n" \
    + "Max Topics: " + str(max_topics) \
    + "\nMax Topics: " + str(max_publishers) \
    + "\nMessages per Topic: " + str(msg_count) \
    + "\nTotal Time: " + str(elapsed_time) \
    + "\nActual rate: " + str(max_topics*msg_count/(elapsed_time)) + " messages/second\n"
    
    #client.publish(topic_summary, message,  qos=QOS, retain=RETAIN)
    print(message)
    
    #client.loop_stop()
    # disconnect publishing thread
    client.disconnect();
    
cmd_fetch_srv_stats = ['ssh', '-t', 'root@192.168.29.149', '/usr/share/mqtt/stats-3.sh']

def execute_shell_cmd(cmd):
    process = subprocess.Popen(cmd, 
                           stdout=subprocess.PIPE,
                           universal_newlines=True)
    buf = ""
    while True:
        output = process.stdout.readline()
        buf += output.strip()
        #msg += "\n"+buf
        #print(msg)
        # Do something else
        return_code = process.poll()
        if return_code is not None:
            #print('buf', 'Success:', return_code)
            # Process has finished, read rest of the output 
            for output in process.stdout.readlines():
                buf += output.strip()
                #print(buf)
                #msg += "\n"+buf
            break
    return buf

file="/home/biswa/mqtt/stats0.csv"
header='\"Reading#,Overall Cpu(%),Mqtt cpu usage(%),Disk Read(kb/s),Disk Write(kb/s),Swap Usage(Bytes),Memory(%),NW Rx(Mbps),NW Tx(Mbps), Latency(ms)\"'
cmd_echo_header = 'echo '+header+' > '+file

i = 0
# on_message call back for latency calculations
def on_message(client, userdata, msg):
    global i
    received_time = timer()
    # check for end of experiment message
    if "end" in msg.payload.decode():
        print("Disconnecting latency calculator !")
        client.publish(topic_summary, "Disconnecting latency calculator !\r",  qos=QOS, retain=RETAIN)
        client.disconnect()
    else:
        latency = received_time - float(msg.payload.decode())
        ret = execute_shell_cmd(cmd_fetch_srv_stats)
        msg = str(i)+","+ret+","+str(latency*1000)
        print(msg)
        cmd_echo_message = 'echo '+msg+' >> '+file
        os.system(cmd_echo_message)
        client.publish(topic_summary, msg+"\r",  qos=QOS, retain=RETAIN) 
        i += 1

# stops latency calculator loop.
def on_disconnect(client, userdata,rc=0):
    client.loop_stop()

# latency calculator thread, sends a message with current timestamp to sample latency.
def latency_calculator(num):
    print("Latency calculator started ...\r")
    os.system(cmd_echo_header)
    latency_calc_sub = mqtt.Client()
    latency_calc_sub.on_message = on_message
    latency_calc_sub.on_disconnect = on_disconnect
    latency_calc_sub.connect(SERVER_IP, PORT, KEEP_ALIVE)
    latency_calc_sub.subscribe(topic_latency, QOS)

    print("Waiting for Publishers to start...\r")
    while(not_end):
        latency_calc_sub.loop_start()
        latency_calc_sub.publish(topic_latency, str(timer())+"\r",  qos=QOS, retain=RETAIN)
        time.sleep(2)

if __name__ == "__main__":
    # parse commandline arguments first.
    #if (len(sys.argv)!= 0 ):
        #usage()
        #sys.exit(2)
        # publisher mqtt client object, send 'end' message to latency client, house keeping
        
    publisher_main = mqtt.Client()
    publisher_main.connect(SERVER_IP, PORT, KEEP_ALIVE)
    # send a preamble
    publisher_main.publish(topic_summary, "start\r",  qos=QOS, retain=RETAIN)
    
    # add a latency sampler process
    print("Registering latency Calculator..\r")
    pl = multiprocessing.Process(target=latency_calculator, args=(0,))
    pl.start()

    time.sleep(6)
    
    # publishers
    proc_list = []  
    for n in range(max_publishers):
        # creating publisher process
        print("Registering Publisher[" + str(n) + "]..\r")
        p = multiprocessing.Process(target=publisher_process, args=(n,))
        proc_list.append(p)
        
    exp_start = timer()
    # schedule all procs
    for proc in proc_list:
        proc.start()

    # joins procs
    for proc in proc_list:
        proc.join()
        
    exp_end = timer()
    
    time.sleep(8)
    # notify triggerer to stop processing
    not_end = 0
        
    # send a message on latency channel to stop processing
    print("Wrapping up latency calculator..\r")
    publisher_main.publish(topic_latency, "end\r",  qos=QOS, retain=RETAIN)
    publisher_main.disconnect()
    
    print("PL\r", pl.is_alive())
    if pl.is_alive():
        pl.terminate()
    
    print("Total number of messages: " + str(msg_count * max_topics * max_publishers) +"\r" \
    + "\nTotal Time: " + str(exp_end - exp_start))
    print("Done !")


Registering latency Calculator..
Latency calculator started ...
Waiting for Publishers to start...
0,0,5.6,22.56,110.96,0,15.372,0,0,3.938005000236444
1,0,0.0,22.56,110.95,0,15.3712,0,0,1.9540499997674488
2,0,0.0,22.56,111.02,0,15.3404,0,0,1.4644700022472534
Registering Publisher[0]..
Registering Publisher[1]..
Registering Publisher[2]..
Publisher [0] started...
Publisher [1] started...
Publisher [2] started...
3,0.980392,5.9,22.55,111.01,0,15.3078,0,0,1.9799479996436276
4,95,368.8,22.55,111.00,0,15.9833,8.69064,10.2444,1.5960480013745837
5,97.9592,400.0,22.55,111.01,0,15.8057,8.80882,9.20887,2.635508997627767
6,97.9592,347.1,22.55,111.00,0,15.9572,8.5569,9.92184,2.0877199967799243
7,98.0392,352.9,22.55,111.00,0,16.0188,7.76291,7.62455,3.9099230016290676
8,95.9596,362.5,22.54,110.99,0,15.8764,6.35838,6.37527,1.6167260000656825
9,97,368.8,22.54,110.98,0,15.8951,8.92189,10.0211,1.1795159989560489
10,95.9184,381.2,22.54,111.15,0,15.9793,8.70388,8.80188,2.145529000699753
11,95.9184,393.3,2